# Mapped differences between simulations 

In [ ]:
chunk_start

In [ ]:
# First identify what is available....

import glob
import os
from IPython.display import Markdown as md
# ability to open pickle
import warnings
warnings.filterwarnings('ignore')
import pickle
#pkg_resources
import pkg_resources
import numpy as np

import pandas as pd
import nctoolkit as nc
from plotnine import *
import os
import molmass
def get_molar_mass(element):
    from molmass import Formula
    f = Formula(element)
    return f.mass

paths = glob.glob("../../data/climatologies/**/**/*.nc")
measures = [os.path.basename(x).split("-")[0] for x in paths]
variables = [os.path.basename(x).split("-")[2] for x in paths]
i_table = 1
i_figure = 1

df_options = pd.DataFrame({"measure": measures, "variable": variables, "path": paths})

ff = "../../sim_dict.pkl"
sim_0_name = "simulation 1"
sim_1_name = "simulation 2"
if os.path.exists(ff):
    sim_dict = pickle.load(open(ff, "rb"))
    sim_0_name = sim_dict["sim0"]
    sim_1_name = sim_dict["sim1"]

measures = [x for x in ["vertical_integration", "top", "vertical_mean", "bottom"] if x in measures]



107 files were created by nctoolkit in prior or current sessions. Consider running deep_clean!
nctoolkit is using Climate Data Operators version 2.4.4


In [ ]:

for mm in set(measures):

    if mm == "vertical_integration":
        md(f"## Spatially aggregated values across the entire water column")
    if mm == "top":
        md(f"## Sea surface")
    if mm == "bottom":
        md(f"## Near-bottom values")
    if mm == "vertical_mean":
        md(f"## Vertical mean values")

    mm_variables = list(set(df_options[df_options["measure"] == mm]["variable"].values))
    df_all = []
    if mm == "vertical_integration":
        md("Total integrated values were calculated for the northwest European Shelf by vertically integrating the values in each grid cell.")  
    if mm == "bottom":
        md("Near-bottom values were calculated by taking the deepest grid cell with a non-missing value in each grid cell.")
    if mm == "vertical_mean":
        md("Vertical mean values were calculated by calculating the vertical average per grid cell, accounting for cell thickness.") 
    if mm == "top":
        md("Sea surface values were calculated by taking the top value in each grid cell.")
    rr_plot = True
    for vv in mm_variables:
        try:
            vv_paths = df_options[(df_options["measure"] == mm) & (df_options["variable"] == vv)]["path"].values
            # path ending with sim_0.nc
            ff1 = [x for x in vv_paths if x.endswith("sim_0.nc")][0]
            ff2 = [x for x in vv_paths if x.endswith("sim_1.nc")][0]
            #ds1 = nc.open_data("/data/proteus1/scratch/rwi/validations/differences/data/climatologies/chlorophyll/vertical_integration/vertical_integration_climatology_chlorophyll_sim_0.nc")
            ds1 = nc.open_data(ff1)
            ds2 = nc.open_data(ff2)
            try:
                ds1.fix_amm7_grid()
            except:
                pass
            try:
                ds2.fix_amm7_grid()
            except:
                pass
            ds1.tmean()
            ds2.tmean()
            ds1_ave = ds1.copy()
            ds2_ave = ds2.copy()
            # subset to sensible bounds
            # ds1.subset(lon = [-19, 9], lat = [42, 63])
            # ds2.subset(lon = [-19, 9], lat = [42, 63])
            # percentage difference
            # check if temperat in name
            if "temperat" not in vv:
                ds1 / ds2
                ds1 - 1
                ds1 * 100
            else:
                ds1 - ds2
            #ds1 - ds2
            the_var= ds1.variables[0]
            values = ds1.to_dataframe()[the_var].values
            values = values[~np.isnan(values)]
            max_value = float(values.max())
            min_value = float(values.min())

            ds1.rename({the_var: "diff"}) 
            # calculate the 95th percentile
            p95 = np.percentile(values, 97.5)
            p95 = min(p95, 100)
            p5 = np.percentile(values, 2.5)
            p5 = max(p5, -100)
            p95 = float(p95)
            p5 = float(p5)
            if max_value < 100:
                p95 = max_value
            if min_value > -100:
                p5 = min_value
            # ds1.assign(diff = lambda x: min(x.diff, p95))
            # make this more detailed
            ds1.assign(diff = lambda x: (x.diff > p95) * p95 + (x.diff < p5) * p5 + (x.diff >= p5) * (x.diff <= p95) * x.diff)
            #ds1.assign(diff = lambda x: max(x.diff, p5))
            # calculate the 5th percentile
            if mm == "top":
                md(f"### Surface {vv}")
            if mm == "bottom":
                md(f"### Near-bottom {vv}")
            if mm == "vertical_mean":
                md(f"### Vertical mean {vv}")
            if mm == "vertical_integration":
                md(f"### Vertically integrated {vv}")
            ds1.plot()
            if "temperat" in vv:
                md(f"**Figure {i_figure}**: Difference between {sim_0_name} and {sim_1_name} for {vv} ({mm}). A positive value indicates that {sim_0_name} is higher than {sim_1_name}.")
            else:
                md(f"**Figure {i_figure}**: Percentage difference between {sim_0_name} and {sim_1_name} for {vv} ({mm}). A positive value indicates that {sim_0_name} is higher than {sim_1_name}.")
            i_figure = i_figure + 1



        except:
            pass

